# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.


In [116]:
#pip install --upgrade langchain

In [117]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [118]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
import pandas as pd

In [119]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [120]:
from langchain.indexes import VectorstoreIndexCreator

In [121]:
#pip install docarray

In [122]:
from langchain.indexes import VectorstoreIndexCreator

In [160]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [161]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [162]:
response = index.query(query)

In [164]:
display(Markdown(response))




| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, 52% polyester and 48% nylon, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, 71% Nylon, 29% Polyester, 100% Polyester knit mesh, wrinkle resistant, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated, 78% nylon, 22% Lycra Xtra Life fiber, wicks moisture, fits comfortably over swimsuit, abrasion resistant |

### In Details

In [127]:
loader = CSVLoader(file_path=file)


In [128]:
docs = loader.load()


In [129]:
docs[0]


Document(page_content=": 0\nUnnamed: 0: 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [130]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [131]:
embed = embeddings.embed_query("Hi my name is Harrison")


In [132]:
print(len(embed))


1536


In [133]:
print(embed[:5])


[-0.021913960576057434, 0.006774206645786762, -0.018190348520874977, -0.039148248732089996, -0.014089343138039112]


In [134]:
# This object create an overall Vectore Store of a list of documents
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [137]:
query = "Please suggest a shirt with sunblocking"

In [139]:
docs = db.similarity_search(query)

In [146]:
len(docs)

4

In [147]:
docs[0]

Document(page_content=': 255\nUnnamed: 0: 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

In [148]:
retriever = db.as_retriever()

In [149]:
llm = ChatOpenAI(temperature = 0.0)

In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")

In [ ]:
display(Markdown(response))

### Automated Way

In [151]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [152]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [153]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [154]:
display(Markdown(response))

| Shirt Name | Description |
|------------|-------------|
| Men's Tropical Plaid Short-Sleeve Shirt | Our lightest hot-weather shirt is rated UPF 50+ for superior protection from the sun's UV rays. Made of 100% polyester, this shirt is wrinkle-resistant and features front and back cape venting and two front bellows pockets. Imported. |
| Men's Plaid Tropic Shirt, Short-Sleeve | This Ultracomfortable sun protection shirt is rated UPF 50+ and is great for extended travel. Made with 52% polyester and 48% nylon, it is wrinkle-free and quickly evaporates perspiration. Features front and back cape venting, two front bellows pockets, and an imported design. |
| Sun Shield Shirt | This high-performance sun shirt is guaranteed to protect from harmful UV rays. Made with 78% nylon and 22% Lycra Xtra Life fiber, it is UPF 50+ rated and wicks moisture for quick-drying comfort. Fits comfortably over swimsuits and is abrasion resistant. |
| Men's TropicVibe Shirt, Short-Sleeve | This Men's sun-protection shirt has built-in UPF 50+ and a lightweight feel. Made with 71% Nylon and 29% Polyester, it is wrinkle resistant and features front and back cape venting and two front bellows pockets. |

Summary:
- Men's Tropical Plaid Short-Sleeve Shirt: Lightest hot-weather shirt with UPF 50+ protection, wrinkle-resistant, and features venting and pockets.
- Men's Plaid Tropic Shirt, Short-Sleeve: Ultracomfortable shirt with UPF 50+ protection, ideal for travel, wrinkle-free, and quick-drying.
- Sun Shield Shirt: High-performance sun shirt with UPF 50+ protection, moisture-wicking, fits over swimsuits, and abrasion resistant.
- Men's TropicVibe Shirt, Short-Sleeve: Sun-protection shirt with UPF 50+ and a lightweight feel, wrinkle-resistant, and features venting and pockets.

In [155]:
response = index.query(query, llm=llm)

In [156]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

In [159]:
display(Markdown(response))

| Shirt Name | Description |
|------------|-------------|
| Men's Tropical Plaid Short-Sleeve Shirt | Our lightest hot-weather shirt is rated UPF 50+ for superior protection from the sun's UV rays. Made of 100% polyester, it is wrinkle-resistant and features front and back cape venting and two front bellows pockets. Imported. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Originally designed for fishing, this shirt offers UPF 50+ coverage and is great for extended travel. Made with 52% polyester and 48% nylon, it is wrinkle-free and quickly evaporates perspiration. Features front and back cape venting, two front bellows pockets, and an imported design. |
| Men's TropicVibe Shirt, Short-Sleeve | This shirt has built-in UPF 50+ sun protection and a lightweight feel. Made with 71% nylon and 29% polyester, it is wrinkle-resistant and features front and back cape venting and two front bellows pockets. Imported. |
| Sun Shield Shirt | This high-performance sun shirt is guaranteed to protect from harmful UV rays. Made with 78% nylon and 22% Lycra Xtra Life fiber, it is UPF 50+ rated and wicks moisture for quick-drying comfort. Fits comfortably over swimsuits and is abrasion-resistant. Imported. |

Summary:
1. Men's Tropical Plaid Short-Sleeve Shirt: The lightest hot-weather shirt with UPF 50+ sun protection, wrinkle-resistant fabric, and front and back venting.
2. Men's Plaid Tropic Shirt, Short-Sleeve: Designed for fishing and extended travel, this shirt offers UPF 50+ coverage, quick-drying fabric, and front and back venting.
3. Men's TropicVibe Shirt, Short-Sleeve: A lightweight shirt with built-in UPF 50+ sun protection, wrinkle-resistant fabric, and front and back venting.
4. Sun Shield Shirt: A high-performance sun shirt with UPF 50+ protection, moisture-wicking fabric, and abrasion resistance. Fits comfortably over swimsuits.